In [1]:
import csv
import numpy as np
import xgboost as xgb
import sklearn
import random
import sklearn.linear_model

/home/ethanid/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
mapper = {}

with open("./states.csv", 'r') as blah:
    for line in csv.reader(blah):
        mapper[line[0]] = line[1]
        
print(mapper)

{'State': 'Abbreviation', 'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS', 'Missouri': 'MO', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


In [3]:
from collections import defaultdict

state_policies = defaultdict(set)

with open("./PolicyBreakdownByState.csv", 'r') as blah:
    policy_lines = list(csv.reader(blah))
    
    legend = policy_lines[0]
    
    for line in policy_lines[1:]:
        for item, typ in zip(line[1:], legend[1:]):
            if item == '1':
                state_policies[typ].add(mapper[line[0]])

print(state_policies)

defaultdict(<class 'set'>, {'Governance': {'CT', 'FL', 'DE', 'RI', 'WV', 'GA', 'LA', 'IL', 'KY', 'MT', 'OH', 'TN', 'AL', 'NH', 'UT'}, 'Monitoring': {'FL', 'LA', 'TX', 'TN', 'VT', 'UT', 'CT', 'RI', 'ID', 'DE', 'MD', 'NY', 'VA', 'WA', 'NJ', 'OH', 'IN', 'NC', 'ME', 'CO', 'CA'}, 'Naloxone': {'FL', 'GA', 'OK', 'LA', 'TX', 'TN', 'UT', 'CT', 'OR', 'DE', 'MD', 'NY', 'WA', 'KY', 'NJ', 'OH', 'ME', 'CO', 'WI', 'CA', 'MN'}, 'Investigation': {'FL', 'VA', 'WV', 'CO', 'MA', 'CA', 'NH', 'UT'}, 'Quantity control': {'ME', 'OK', 'IL', 'CO', 'MT', 'OH', 'MA', 'UT'}, 'Education': {'OR', 'ME', 'IL', 'TX', 'NM', 'MA', 'OH', 'ID', 'UT'}})


In [4]:
SURPRESSED_DEATH_INPUTATION = 0

county_death_rate = {}

with open('./deathData.txt', 'r') as foo:
    lines = list(csv.reader(foo, delimiter='\t'))
    
    header = lines[0]
    
    for line in lines[1:]:
        if line[0] == '---':
            break
        _, name, number, year, year_number, deaths, population, _ = line
        if year != '2016':
            continue

        if deaths == 'Missing':
            pass
        elif deaths == 'Suppressed':
            county_death_rate[name] = SURPRESSED_DEATH_INPUTATION
        else:
            county_death_rate[name] = int(deaths) / int(population)



In [5]:
with open("./2014 CHR analytic data.csv", 'r') as foo:
    lines = list(csv.reader(foo))
    
first = lines[0]
second = lines[1]

fields = [(i, name, desc) for i, (name, desc) in enumerate(zip(first, second)) if name.startswith('measure_') and name.endswith('value')]

county_lines = lines[2:]
random.shuffle(county_lines)

In [44]:
data = []
meta_labels = []
death_labels = []
final_labels = defaultdict(list)
        
for line in county_lines:
    state, county = line[2], line[3]
    full_county = '{}, {}'.format(county, state)
    
    if full_county not in county_death_rate:
        continue
    data_row = []
    for i, _, _ in fields:
        text_value = line[i]
        if text_value == '':
            data_row.append(np.nan)
        else:
            data_row.append(float(text_value.replace(',', '')))
    data.append(data_row)
    meta_labels.append((state, county))
    death_labels.append(county_death_rate[full_county])
    
    for policy, set_good in state_policies.items():
        if state in set_good:
            final_labels[policy].append(1)
        else:
            final_labels[policy].append(0)
        
data = np.array(data)

In [57]:
def compute_estimates(X, labels, death):
    shuffle_X, shuffle_labels = sklearn.utils.shuffle(X, labels)
    
    param_grid = {
        'C': [10**(a) for a in range(-5, 4)],
    }

    estimator = sklearn.linear_model.LogisticRegression()
    
    imputer = sklearn.preprocessing.Imputer()
    shuffle_X = imputer.fit_transform(shuffle_X)

    scaler = sklearn.preprocessing.StandardScaler()
    shuffle_X = scaler.fit_transform(shuffle_X)

    trained_model = sklearn.model_selection.GridSearchCV(
            estimator=estimator,
            param_grid=param_grid, 
            scoring='roc_auc', 
            verbose=1,
            n_jobs=1,
            refit=True,
            cv=5,
    )

    trained_model.fit(shuffle_X, shuffle_labels)

    print(trained_model.best_params_)
    print(trained_model.best_score_)
        
    estimated_effect = 0

    final_probs = trained_models[policy].predict_proba(X)
    for prob_score, label, final_deaths in zip(final_probs[:, 1], labels, death):
        if label == 1:
            estimated_effect += final_deaths / prob_score 
        else:
            estimated_effect -= final_deaths / (1 - prob_score)

    estimated_effect /= len(death)
    return estimated_effect * 100 * 1000

In [ ]:
point_estimates = {}
bootstrap_estimates = {}

for policy, labels in final_labels.items():
    print(policy)
    point_estimate = compute_estimates(data, labels, death_labels)
    
    print(point_estimate)
    point_estimates[policy] = point_estimate
    
    bootstraps = []
    for i in range(20):
        sub_X, sub_labels, sub_death = sklearn.utils.resample(data, labels, death_labels)
        bootstraps.append(compute_estimates(sub_X, sub_labels, sub_death))
    bootstrap_estimates[policy] = bootstraps
    print(bootstraps)
        

Governance
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    4.3s finished


{'C': 1000}
0.883304782026
-1.43612239872
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    5.8s finished


{'C': 1000}
0.894979853469
Fitting 5 folds for each of 9 candidates, totalling 45 fits


...

{'C': 1000}
0.892311089313
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    5.6s finished


{'C': 100}
0.892239738372
Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    4.6s finished


{'C': 1000}
0.896643474694
Fitting 5 folds for each of 9 candidates, totalling 45 fits
